In [1]:
# import numpy as np
# import uuid
# import json
# from copy import deepcopy
# from IPython.display import Javascript,HTML
import pandas as pd

# from collections import OrderedDict

In [2]:
from krisk import init_notebook

In [3]:
init_notebook()

<IPython.core.display.Javascript object>

In [4]:
import krisk.plot as plt

In [5]:
df = pd.read_csv('krisk/tests/data/gapminderDataFiveYear.txt', sep='\t')

In [6]:
a = plt.bar(df,'continent',y='lifeExp',how='mean',stacked=True,annotate=True)

In [7]:
a

In [14]:
from krisk.connections import get_paths

In [15]:
get_paths()

odict_keys(['echarts'])

In [13]:
print(p._get_resync_option_strings({}))


require(['echarts'],function(echarts){
    
    function parseFunction(str){
        return eval('(' + str + ')');
    }
    
    var myChart = echarts.init(document.getElementById("35501550-9868-4a02-926e-840820724705"),"");
    
    var option = {};
    option['tooltip']['formatter'] = parseFunction(option['tooltip']['formatter']);
    
    //option['series'][0]['symbolSize'] = function (val){return val[2]*10;}
    
    
    myChart.setOption(option);
    //console.log(option);
    
    
});



In [ ]:
# %load krisk/template.py

def save_html(script,path):
    from jinja2 import Template
    html_template = open('template.html','r')
    script = script.replace('element','$("body")')
    f = open(path,'w')
    f.write(Template(html_template.read()).render(SCRIPT=script))
    f.close()
    html_template.close()


RESET_OPTION = """
require({requires},function(echarts){{
    
    function parseFunction(str){{
        return eval('(' + str + ')');
    }}
    
    var myChart = echarts.init(document.getElementById("{chartId}"),"{theme}");
    
    var option = {option};
    option['tooltip']['formatter'] = parseFunction(option['tooltip']['formatter']);
    
    //option['series'][0]['symbolSize'] = function (val){{return val[2]*10;}}
    
    
    myChart.setOption(option);
    //console.log(option);
    {events}
    
}});
"""


APPEND_ELEMENT = """
$('#{id}').attr('id','{id}'+'_old');
element.append('<div id="{id}" style="width: 600px;height:400px;"></div>');"""


OPTION_TEMPLATE = {
        'title': {
            'text': ''
        },
        'tooltip': {'axisPointer':{'type':''}},
        'legend': {
            'data':[]
        },
        'xAxis': {
            'data': []
        },
        'yAxis': {},
        'series': []
    }


EVENTS_TEMPLATE = """
myChart.on('{event}',function(params){{

    var d_params  = {{'series':{{'name':params.seriesName,
                                 'index':params.seriesIndex}},
                      'data':{{'value':params.value,
                               'index':params.dataIndex,
                               'name':params.name}}
    }}
    
    console.log('parameters extracted: ');
    console.log(d_params);
    
    // Create new cell and execute function passed with parameters
    var nb = Jupyter.notebook;
    nb.insert_cell_below();
    nb.select_next();
    
    var json_strings = JSON.stringify(d_params);

    var cell = nb.get_selected_cell();
    var code_input = "{function}(json.loads('" + json_strings + "'))";
    console.log("Executing code: " + code_input);
    cell.set_text(code_input);
    cell.execute();
    
    
    // Immediately delete the cell after execute
    nb.delete_cell();
    
}});
"""

In [8]:
p.to_html('../chart_scatter2.html')

In [9]:
pwd

'/Volumes/Marice-Dev/Testing/echarts/pandas-echarts'

In [9]:
# def init_notebook():
#     return Javascript("""
#             require.config({
#                   paths: {
#                       echarts: '//echarts.baidu.com/dist/echarts.min',
#                       dark: '//echarts.baidu.com/asset/theme/dark',
#                       vintage: '//echarts.baidu.com/asset/theme/vintage',
#                       roma: '//echarts.baidu.com/asset/theme/roma',
#                       shine: '//echarts.baidu.com/asset/theme/shine',
#                       infographic: '//echarts.baidu.com/asset/theme/infographic',
#                       macarons: '//echarts.baidu.com/asset/theme/macarons'
                      
#                   }
#                 });""")

In [7]:
p = scatter(df[df.year == 2007],'lifeExp','pop')
p.set_tooltip_format(['country'])

NameError: name 'scatter' is not defined

In [25]:
p.to_html('chart-me3.html')

In [ ]:
# %load krisk/chart.py

import uuid
import json
from copy import deepcopy
from krisk.template import *
from krisk.connections import get_paths

paths = get_paths()

class Chart():
    def __init__(self,**kwargs):
        self._chartId = str(uuid.uuid4())
        self._option = deepcopy(OPTION_TEMPLATE)    
        self._kwargs_chart_ = kwargs
        self._theme = ''
        
        
    # Color and Themes
    
    def set_theme(self,theme):
        """
        Set the theme of the chart.
        
        Parameters
        ----------

        theme: str
            {'dark','vintage','roma','shine','infographic','macarons'}, default None
        """
        
        
        if theme not in THEMES:
            raise AssertionError("Invalid theme name: {theme}".format(theme=theme))
            
        
        self._theme = theme
        return self
    
    
    def set_color(self,background='',palette=''):
        """
        Set background and pallete color
        
        Parameters
        ----------
        
        background: string
            hex color
        palettes: list of strings
            list hex colors
            
        Returns
        -------
        Chart Object
        """
        
#         TODO:
#         (p
#         .set_color(background='something')
#         .set_color(palettes=[something])) override background to None
        
#         Is this intended? Or should just these parameters made as separate methods?
        
        self._option.pop('color',None)
        self._option.pop('graph',None) #Need further analyze graph color
        self._option.pop('backgroundColor',None)
        
        if background:
            self._option['backgroundColor'] = background
        if palettes:
            self._option['color'] = palettes
            self._option['graph'] = {'color':palettes}
        
        
        return self
        
        
        
    # ---------------------------------------------------------------------------
    
    # Tooltip
    
    def set_tooltip_style(self,trigger='item',axis_pointer='line',trigger_on='mousemove',
                          font_style='normal',font_family='sans-serif',font_size=14):
        
        """Set Tooltip options.
        
        Parameters
        ----------
        trigger: {'item',axis}, default 'item'
            When tooltip should be triggered. Default to item
        axis_pointer: {'shadow','cross','line'}, default 'line'
            Effect of pointing the axis.
        trigger_on: {'mousemove','click'}, default 'mousemove'
            Tooltip trigger
        font_style: string hex, default 'normal'
            Font Style
        font_family: sting, default to 'sans-serif'.
            Tooltip font familty
        font_size: int, default 14.
            Tooltip font size
        
        """
        
        
        self._option['tooltip']['trigger'] = trigger
        self._option['tooltip']['axisPointer']['type'] = axis_pointer
        self._option['tooltip']['triggerOn'] = trigger_on
        
        self._option['tooltip']['fontStyle'] = font_style
        self._option['tooltip']['fontFamily'] = font_family
        self._option['tooltip']['fontSize'] = font_size
        
        return self
    
    
    def set_tooltip_format(self,columns,override=False,
                           formatter = "'{key}' + '：' + {value} + '{unit}' +'<br>'"):
        """
        Set tooltip format. Currently only Scatter plot supported because it's the only that keep the
        data as is.
        
        Parameters
        ----------
        
        columns: list of string or list of tuples
            if list of strings, retrieve the columns value for the tooltip
            if list of tuples, will be represented as key,unit for the format
        override: Boolean, default to False
            provide custom Javascript function
        formatter: string,
            Format key,value,unit that will be rendered in the tooltip
        
        Returns
        -------
        Chart Object
        
        Examples
        --------
        
        """
        
        if self._kwargs_chart_['type'] != 'scatter':
            raise TypeError('Chart Type not supported')
        else:
            f_columns = []
            for c in columns:
                if isinstance(c,str):
                    key,unit =c, ' '
                elif isinstance(c,tuple):
                    key,unit = c
                else:
                    raise TypeError('Columns type not supported')

                idx = self._kwargs_chart_['columns'].index(key)
                f_columns.append(formatter
                                 .format(key=key,
                                         value='value[{idx}]'.format(idx=idx),
                                         unit=unit))

            formatter_strings =  """function (obj) {{
                                    var value = obj.value;
                                    return {f_columns};
                                }}""".format(f_columns='+'.join(f_columns))

            self._option['tooltip']['formatter'] = formatter_strings
            

            return self
    
    # ----------------------------------------------------------------------
    
    def get_option(self):
        """Return Chart option that will be injected to Option Javascript object"""

        return self._option
    
    
    def set_title(self,title,x_pos='auto',y_pos='auto'):
        """Set title for the plot.
        
        The coordinate is started at bottom left corner. If x_pos and y_pos started
        at negative values, then the coordinate started at upper right corner.
        
        Parameters
        ----------
        title: str
            Title of the chart.
        x_pos: str, {'auto', left', 'center', 'right', 'i%'}, default to 'auto'
        y_pos: str, {'auto', top', 'center', 'bottom', 'i%'}, default to 'auto'
        
        """
        
        self._option['title']['text'] = title
        
        if x_pos.startswith('-'):
            self._option['title']['right'] = x_pos[1:]
        else:
            self._option['title']['left'] = x_pos
            
        if y_pos.startswith('-'):
            self._option['title']['top'] = y_pos[1:]
        else:
            self._option['title']['bottom'] = y_pos
        
        
        return self
    
    
    def set_legend(self,align='auto',orient='horizontal',
                   x_pos='auto',y_pos='auto'):
        """
        Set legend style.
        
        The coordinate is started at bottom left corner. If x_pos and y_pos started
        at negative values, then the coordinate started at upper right corner.
        
        Parameters
        ----------
        
        align: str, {'auto','left','right'}, default to 'auto'
        orient: str, {'horizontal','vertical'} default to 'horizontal'
        x_pos: str, {'auto', left', 'center', 'right', 'i%'}, default to 'auto'
        y_pos: str, {'auto', top', 'center', 'bottom', 'i%'}, default to 'auto'
        
        
        Returns
        -------
        Chart Object
        
        
        """
        
        self._option['legend']['align'] = align
        self._option['legend']['orient'] = orient
        
        if x_pos.startswith('-'):
            self._option['legend']['right'] = x_pos[1:]
        else:
            self._option['legend']['left'] = x_pos
            
        if y_pos.startswith('-'):
            self._option['legend']['top'] = y_pos[1:]
        else:
            self._option['legend']['bottom'] = y_pos
            
            
        return self
    
    
    def flip_axes(self):
        """Flip the axes to make it horizontal"""
        
        self._axes_swapped = not self._axes_swapped
        self._option['xAxis'],self._option['yAxis'] = self._option['yAxis'],self._option['xAxis']
        return self
    
    # Events
    def on_event(self,event,handler):
        """
        Parameter:
        event: {'click','dblclick','mousedown','mouseup','mouseover','mouseout','globalout'}, default None
            In which event the function should be triggered
        handler: function
            The trigger function
        """
        
        events = ['click','dblclick','mousedown','mouseup','mouseover','mouseout','globalout']
        if event not in events:
            raise AssertionError('Invalid event name: %s'% event)
            
        self._events[event] = handler.__name__
        return self
    
    
    
    # --------------------------------------------------------------------------
    
    # Replot Functions
    def resync_data(self,data):
        """
        Update data but still using the same chart option.
        Currently just update the current cell it exist, but not the chart option
        itself.
        
        Parameters
        ----------
        data: pd.DataFrame
         
        """
        option = make_chart(data,**self._kwargs_chart_)._option
        return Javascript(self._get_resync_option_strings(option))
    
    def replot(self,chart):
        """Replot entire chart to its current cell"""
        return Javascript(self._get_resync_option_strings(chart._option))
    
    def _get_resync_option_strings(self,option):
        """Resync Chart option"""
        
        events = [EVENTS_TEMPLATE.format(event=e,function=self._events[e]) for e in self._events]
        OPTION_KWS = dict(
            requires=list(paths).__repr__(),
            chartId=self._chartId,
            theme=self._theme,
            option=json.dumps(option,indent=4),
            events='\n'.join(events)
        )
        return RESET_OPTION.format(**OPTION_KWS)
    
    
    def _repr_javascript_(self):
        """Embedding the result of the plot to Jupyter"""
        return (APPEND_ELEMENT.format(id=self._chartId))+\
                (self._get_resync_option_strings(self._option))
    # ----------------------------------------------------------------------
    
    # Saving chart option
    def to_json(self,path):
        "Save Chart option"
        
        json.dump(self._option,open(path,'w'))
        
    
    def to_html(self,path):
        "Save full html file"
        save_html(self._repr_javascript_(),path)
    
    
    _axes_swapped = True
    _kwargs_chart_ = {}
    _events = {}
        
    

In [254]:
df['Pop'] = df['pop'] / 1000000

In [271]:
df['pop'].max()

1318683096.0

In [272]:
df['Pop'].max()

1318.683096

In [277]:
p.to_html('scatter3.html')

In [276]:
p = scatter(df[df.year == 2007],'lifeExp','gdpPercap',size='pop')
# p.set_tooltip_format(['country','lifeExp','continent'])

In [235]:
# %%file pandas-echarts/tests/test_option.py


df = pd.read_csv('gapminderDataFiveYear.txt',sep='\t')

def test_bar():
    
    true_option = json.load(open('pandas-echarts/tests/data/bar_option.json','r'))
    p = bar(df,'lifeExp',y='pop',category='continent',how='mean',stacked=True)
    
    assert p._option == true_option
    
def test_line():
    
    true_option = json.load(open('pandas-echarts/tests/data/line_option.json','r'))
    p = line(df,'year',y='lifeExp',category='continent',how='mean',stacked=True,area=True)
    
    assert p._option == true_option
    
def test_hist():
    
    true_option = json.load(open('pandas-echarts/tests/data/hist_option.json','r'))
    p = hist(df,'lifeExp',category='continent',bins=100,normed=True,stacked=True)
    
    assert p._option == true_option
    
def test_scatter():
    
    true_option = json.load(open('pandas-echarts/tests/data/scatter_option.json','r'))
    p = scatter(df[df.year == 1952],'lifeExp','gdpPercap',size='pop',category='continent')
    
    assert p._option == true_option

test_bar()
test_line()
test_hist()
test_scatter()

AssertionError: 

In [39]:
from bokeh import colors
from bokeh import palettes

In [23]:
true_option = json.load(open('pandas-echarts/tests/data/scatter_option.json','r'))

In [29]:
d = {}

In [30]:
d['label'] = {'name':'hellow'}

In [131]:
def foo(params):
    
    d.resync_data(df)
    return "Hello World"

In [132]:
d = bar(df,x='continent')
d.on_event('mouseover',foo)

In [48]:
ipywidgets.__version__

'4.1.1'

In [85]:
# %%file krisk/tests/test_make_chart.py
import pytest

@pytest.fixture
def chart_gapminder():
    import pandas as pd
    #from krisk import make_chart
    df = pd.read_csv('../echarts/gapminderDataFiveYear.txt',sep='\t')
    return (line(df,x='year',category='continent',y='lifeExp',how='mean',area=True,stacked=True)
             .set_theme('vintage')
             .set_tooltip())


def test_option(chart_gapminder):
    import json
    json_path ='krisk/tests/json_gapminder_line_year_continent_lifeExp_mean_area_stacked.json'
    option = json.load(open(json_path,'r'))
    assert option == chart_gapminder._option
    
test_option(chart_gapminder())

# TODO

Priority:

1. Legend and Title DONE
2. Tooltip DONE? Scatter

to_html DONE

set_toolbox

set_figsize()

refactor insert series DONE?

set item_style

self._data_columns = df.columns
if update:
    self._data.columns != df.columns

set annotate = True DONE

set ticks DONE?

set_color(background=None,palletes=None) DONE?

 set_title(title,pos=('center','top')/'left and set_legend

    'best'         : 0, (only implemented for axes legends)
    'upper right'  : 1,
    'upper left'   : 2,
    'lower left'   : 3,
    'lower right'  : 4,
    'right'        : 5,
    'center left'  : 6,
    'center right' : 7,
    'lower center' : 8,
    'upper center' : 9,
    'center'       : 10,

## bar
* http://echarts.baidu.com/demo.html#bar-y-category DONE
* http://echarts.baidu.com/demo.html#bar-y-category-stack DONE

## scatter
* http://echarts.baidu.com/demo.html#scatter-aqi-color
* http://echarts.baidu.com/demo.html#scatter-world-population
* http://echarts.baidu.com/demo.html#scatter-life-expectancy-timeline

## line
* http://echarts.baidu.com/demo.html#dynamic-data2 DONE
* http://echarts.baidu.com/demo.html#line-stack DONE
* http://echarts.baidu.com/demo.html#area-stack DONE

## Histogram hist with category

* http://echarts.baidu.com/demo.html#bar-animation-delay
* http://echarts.baidu.com/demo.html#mix-line-bar DONE
* http://echarts.baidu.com/demo.html#bar1(np.histogram, lose the markline and markpoint) DONE

### POTENTIALLY BUG

* callback for loading echarts

# Testing Echarts

In [193]:
%%javascript
$('#main').remove()
element.append('<div id="main" style="width: 600px;height:400px;"></div>');
require(['echarts'],function(echarts){
// 基于准备好的dom，初始化echarts实例
    
        var colorPalette = ['#d87c7c','#919e8b', '#d7ab82',  '#6e7074','#61a0a8','#efa18d', '#787464', '#cc7e63', '#724e58', '#4b565b'];
        echarts.registerTheme('vintage', {
            color: colorPalette,
            backgroundColor: '#fef8ef',
            graph: {
                color: colorPalette
            }
        });

        var myChart = echarts.init(document.getElementById('main'),'dark');

        // 指定图表的配置项和数据
        var option = {
            title: {
                text: 'ECharts 入门示例'
            },
            tooltip: {},
            legend: {
                data:['销量']
            },
            xAxis: {
                data: ["衬衫","羊毛衫","雪纺衫","裤子","高跟鞋","袜子"]
            },
            yAxis: {},
            series: [{
                name: '销量',
                type: 'bar',
                data: [5, 20, 36, 10, 10, 20]
            }]
        };

        // 使用刚指定的配置项和数据显示图表。
        myChart.setOption(option);
});

"Hello World"

<IPython.core.display.Javascript object>

In [ ]:
# import json
# option = {
#         'title': {
#             'text': 'ECharts 入门示例'
#         },
#         'tooltip': {},
#         'legend': {
#             'data':['销量']
#         },
#         'xAxis': {
#             'data': ["衬衫","羊毛衫","雪纺衫","裤子","高跟鞋","袜子"]
#         },
#         'yAxis': {},
#         'series': [{
#             'name': '销量',
#             'type': 'bar',
#             'data': [20, 20, 36, 10, 10, 20]
#         }]
#     }
# # Javascript("""var option={}""".format(json.dumps(option)))

In [23]:
from bokeh.io import output_notebook